In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cmc_utils as cmc
import cmctools.cmctoolkit as cmc_tk
import hvss_utils as hvss

#plt.style.use(cmc.MPL_STYLE_FILE_PATH) ## This doesn't work because tex is not installed on vera

In [2]:
### TODO list: make input/output stacked bar plot
# x-axis: initial configurations ((S,S)+C, etc.)
# y-axis: number of encounters
# stacked: outcome (star ejected, CO ejected, ionization, merger)

In [3]:
# Hard-coded [type_i, type_f] matrix for straightforward result identification
# -1 is error value (result isn't possible given type_i)
initials = [t.id_string for t in hvss.BINSINGLE_TYPES_CMC_I]
results = ["Ionization", "Star Ejection", "CO Ejection", "Merger"]
types_to_result = [
    [], # dummy row since type_i starts at 1
    [0, -1, 2, 1, 3],
    [0, -1, 2, 1, 3],
    [0, 2, 1, -1, 3],
    [0, 2, 1, -1, 3],
]

In [4]:
### Make plots
S_SCAT = 0.5

# Get cluster names
cmc_cluster_list = [
    "N8e5_rv0.5_rg8_Z0.0002",
#     "N8e5_rv1_rg8_Z0.0002",
#     "N8e5_rv2_rg8_Z0.0002",
#     "N8e5_rv4_rg8_Z0.0002",
]

N = 10

for ci, cmc_cluster in enumerate(cmc_cluster_list):
    print("{}, N={}".format(cmc_cluster, N))

    # File names and directory creation
    data_file_path = (
        hvss.DATA_PATH
        + cmc_cluster
        + "/{}_N-{}".format(hvss.FB_OUTPUT_FILENAME, N)
    )

    # Read saved data file
    print("\tLoading data...")
    singles = pd.read_csv(
        data_file_path + "_singles.txt",
        names=hvss.SINGLES_HEADERS,
        dtype=np.float64,
    )
    
    # Convert types to ints
    singles.type_i = singles.type_i.astype(int)
    singles.type_f = singles.type_f.astype(int)
    
    # Get the counts we want
    df_vc = singles.value_counts(subset=["type_i","type_f",],sort=False).to_frame(name="Counts")
    del singles
    
    # Classify results
    df_vc["Result"] = 0
    for ri, r in df_vc.iterrows():
        r["Result"] = types_to_result[ri[0]][ri[1]]
        
    ions = df_vc[df_vc.Result == 0]["Counts"].values
    stes = df_vc[df_vc.Result == 1]["Counts"].values
    coes = df_vc[df_vc.Result == 2]["Counts"].values
    mrgs = df_vc[df_vc.Result == 3]["Counts"].values
    
    sbar = plt.bar(initials, stes, label="Star Ejection", color="xkcd:orangered")
    cbar = plt.bar(initials, coes, bottom=stes, label="CO Ejection", color="xkcd:azure")
    ibar = plt.bar(initials, ions, bottom=stes+coes, label="Ionization", color="xkcd:goldenrod")
    mbar = plt.bar(initials, mrgs, bottom=stes+coes+ions, label="Merger", color="xkcd:violet")
    plt.legend()
    
    
    # Add percentage labels to bar plot.
    # This does not work yet - the series/1D arrays for each outcome do not have information about the size of the total configuration bar
    def autolabel(rects):
        total = sum([r.get_height() for r in rects])
        for idx,rect in enumerate(rects):
            height = rect.get_height()
            if height > 10000:
                plt.text(rect.get_x() + rect.get_width()/2., rect.get_y() + height/2.,
                        "%.1f%%" % (height/total*100),
                        ha='center', va='center', rotation=0)
    autolabel(sbar)
    autolabel(cbar)
    autolabel(ibar)
    autolabel(mbar)
df_vc

N8e5_rv0.5_rg8_Z0.0002, N=10
	Loading data...


Counts  Result
type_i type_f                
1      0          969       0
       2          958       2
       3       244097       1
       4          358       3
2      0        73989       0
       2       124405       2
       3        14026       1
       4         2934       3
3      0         2544       0
       1          477       2
       2       730266       1
       4         1474       3
4      0         1173       0
       1         9612       2
       2         8021       1
       4         5922       3

RuntimeError: Failed to process string with tex because dvipng could not be found

<Figure size 330x300 with 1 Axes>